<a href="https://colab.research.google.com/github/flahbocchino/farmtech-fase7-grupoFlavia-/blob/main/FarmTech_Fase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests


In [2]:
# Dados de exemplo - você pode adaptar para seus dados reais
dados_talhoes = {
    "talhao_id": [1, 2, 3],
    "cultura": ["Soja", "Milho", "Cana"],
    "comprimento_m": [100, 150, 200],
    "largura_m": [50, 80, 120],
}

df_talhoes = pd.DataFrame(dados_talhoes)
df_talhoes


,talhao_id,cultura,comprimento_m,largura_m
0,1,Soja,100,50
1,2,Milho,150,80
2,3,Cana,200,120


In [3]:
# Cálculo da área em m²
df_talhoes["area_m2"] = df_talhoes["comprimento_m"] * df_talhoes["largura_m"]

# Conversão para hectares (1 ha = 10.000 m²)
df_talhoes["area_ha"] = df_talhoes["area_m2"] / 10_000

df_talhoes


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha
0,1,Soja,100,50,5000,0.5
1,2,Milho,150,80,12000,1.2
2,3,Cana,200,120,24000,2.4


In [4]:
# Dose de fertilizante em kg por hectare, por cultura
fertilizante_por_cultura = {
    "Soja": 50,
    "Milho": 70,
    "Cana": 90
}

def calcula_fertilizante(cultura, area_ha):
    # pega a dose por hectare da cultura; se não achar, usa 50 como padrão
    dose_ha = fertililizante_por_cultura.get(cultura, 50)
    return dose_ha * area_ha

# Atenção: corrige o nome do dicionário (erro de digitação)
fertilizante_por_cultura


{'Soja': 50, 'Milho': 70, 'Cana': 90}

In [5]:
# Dose de fertilizante em kg por hectare, por cultura
fertilizante_por_cultura = {
    "Soja": 50,
    "Milho": 70,
    "Cana": 90
}

def calcula_fertilizante(cultura, area_ha):
    # pega a dose por hectare da cultura; se não achar, usa 50 como padrão
    dose_ha = fertilizante_por_cultura.get(cultura, 50)
    return dose_ha * area_ha

# Aplica a função linha a linha
df_talhoes["fertilizante_kg"] = df_talhoes.apply(
    lambda row: calcula_fertilizante(row["cultura"], row["area_ha"]),
    axis=1
)

df_talhoes


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg
0,1,Soja,100,50,5000,0.5,25.0
1,2,Milho,150,80,12000,1.2,84.0
2,3,Cana,200,120,24000,2.4,216.0


In [6]:
# Salvar base de talhões com área e insumos
df_talhoes.to_csv("base_fase1_talhoes_insumos.csv", index=False)

df_talhoes


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg
0,1,Soja,100,50,5000,0.5,25.0
1,2,Milho,150,80,12000,1.2,84.0
2,3,Cana,200,120,24000,2.4,216.0


In [7]:
# Coordenadas de exemplo (pode imaginar a fazenda em SP interior)
latitude = -22.5
longitude = -47.5

url = (
    "https://api.open-meteo.com/v1/forecast"
    "?latitude={lat}&longitude={lon}"
    "&hourly=temperature_2m,precipitation"
    "&past_days=7"
    "&timezone=auto"
).format(lat=latitude, lon=longitude)

response = requests.get(url)
dados_meteo = response.json()

dados_meteo.keys()


dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly'])

In [8]:
horas = dados_meteo["hourly"]["time"]
temperaturas = dados_meteo["hourly"]["temperature_2m"]
chuvas = dados_meteo["hourly"]["precipitation"]

df_meteo_horario = pd.DataFrame({
    "time": pd.to_datetime(horas),
    "temperatura_C": temperaturas,
    "chuva_mm": chuvas
})

df_meteo_horario.head()


,time,temperatura_C,chuva_mm
0,2025-11-15 00:00:00,21.1,0.0
1,2025-11-15 01:00:00,20.5,0.0
2,2025-11-15 02:00:00,19.9,0.0
3,2025-11-15 03:00:00,19.4,0.0
4,2025-11-15 04:00:00,19.1,0.0


In [9]:
df_meteo_diario = (
    df_meteo_horario
    .set_index("time")
    .resample("D")
    .agg({
        "temperatura_C": "mean",
        "chuva_mm": "sum"
    })
    .reset_index()
)

df_meteo_diario


,time,temperatura_C,chuva_mm
0,2025-11-15,23.654167,0.4
1,2025-11-16,21.820833,7.8
2,2025-11-17,23.533333,1.8
3,2025-11-18,22.420833,3.6
4,2025-11-19,21.308333,0.0
5,2025-11-20,21.466667,0.0
6,2025-11-21,24.033333,0.0
7,2025-11-22,25.754167,0.1
8,2025-11-23,22.650000,13.4
9,2025-11-24,20.462500,13.8


In [10]:
temp_media_7d = df_meteo_diario["temperatura_C"].mean()
chuva_total_7d = df_meteo_diario["chuva_mm"].sum()

temp_media_7d, chuva_total_7d


(np.float64(22.423214285714284), np.float64(40.9))

In [11]:
df_talhoes["temp_media_7d"] = temp_media_7d
df_talhoes["chuva_total_7d"] = chuva_total_7d

df_talhoes


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg,temp_media_7d,chuva_total_7d
0,1,Soja,100,50,5000,0.5,25.0,22.423214,40.9
1,2,Milho,150,80,12000,1.2,84.0,22.423214,40.9
2,3,Cana,200,120,24000,2.4,216.0,22.423214,40.9


In [12]:
def recomenda_irrigacao(chuva_total_7d):
    if chuva_total_7d < 10:
        return "Alta prioridade de irrigação"
    elif chuva_total_7d < 30:
        return "Monitorar irrigação"
    else:
        return "Irrigação reduzida"

df_talhoes["recomendacao_irrigacao"] = recomenda_irrigacao(chuva_total_7d)

df_talhoes


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg,temp_media_7d,chuva_total_7d,recomendacao_irrigacao
0,1,Soja,100,50,5000,0.5,25.0,22.423214,40.9,Irrigação reduzida
1,2,Milho,150,80,12000,1.2,84.0,22.423214,40.9,Irrigação reduzida
2,3,Cana,200,120,24000,2.4,216.0,22.423214,40.9,Irrigação reduzida


In [13]:
df_talhoes.to_csv("base_fase1_com_meteo.csv", index=False)
df_talhoes.head()


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg,temp_media_7d,chuva_total_7d,recomendacao_irrigacao
0,1,Soja,100,50,5000,0.5,25.0,22.423214,40.9,Irrigação reduzida
1,2,Milho,150,80,12000,1.2,84.0,22.423214,40.9,Irrigação reduzida
2,3,Cana,200,120,24000,2.4,216.0,22.423214,40.9,Irrigação reduzida
